In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import shapely
from shapely.geometry import Point, LineString, MultiLineString, Polygon, MultiPolygon
from tqdm.notebook import tqdm
import time
from datetime import datetime

import momepy as mm
import networkx as nx

import math

from contextlib import redirect_stdout

import io
import sys

from shapely.ops import nearest_points
from shapely.ops import unary_union

In [ ]:
# проверка остановки в буффере - если даже по вохдуху не попадает - не искать

In [2]:
graph = gpd.read_file('../lines_to_graph/with_ped/new_graph_with_ped.json',
                        encoding='utf-8')

In [3]:
nodes = gpd.read_file('../lines_to_graph/with_ped/all_nodes_with_ped.json',
                        encoding='utf-8')

In [4]:
buildings = gpd.read_file('./all_bui.json',encoding='utf-8')

In [5]:
stps_big = pd.read_csv('../lines_to_graph/total/df_stop_seq_all_n_samobl.csv',
                        encoding='utf-8', sep=';')

In [6]:
stps_big = stps_big[['stop_id', 'stop_name', 'lat', 'lon']]
stps_big = stps_big.drop_duplicates(subset=['stop_id']).reset_index(drop=True)

In [7]:
lst_geo=[]
lst_lat=list(stps_big.lat)
lst_lon=list(stps_big.lon)
i=0
for i in tqdm(range(len(stps_big))):
    one_pt = Point(lst_lon[i], lst_lat[i])
    lst_geo.append(one_pt)
# 

In [8]:
gdf_stops_all = gpd.GeoDataFrame(stps_big, geometry=lst_geo)
gdf_stops_all.crs='epsg:4326'

In [9]:
border_sa = gpd.read_file('./data/outer_border_SA.shp',encoding='utf-8')

In [10]:
gdf_stops = gpd.sjoin(gdf_stops_all, border_sa[['geometry']], how='inner', op='intersects')
gdf_stops = gdf_stops.drop('index_right', axis=1).reset_index(drop=True)

In [11]:
# убрать жд пути из графа
graph_ped = graph[graph.type_ped != 'no_ped'].reset_index(drop=True)[['link_id', 'geometry']]

In [12]:
shrt_bui = buildings[['osm_id', 'geometry']].copy()

In [13]:
def intersect_using_spatial_index(source_gdf, intersecting_gdf):
    """
    Conduct spatial intersection using spatial index for candidates GeoDataFrame to make queries faster.
    Note, with this function, you can have multiple Polygons in the 'intersecting_gdf' and it will return all the points
    intersect with ANY of those geometries.
    """
    source_sindex = source_gdf.sindex
    possible_matches_index = []

    # 'itertuples()' function is a faster version of 'iterrows()'
    for other in intersecting_gdf.itertuples():
        bounds = other.geometry.bounds
        c = list(source_sindex.intersection(bounds))
        possible_matches_index += c

    # Get unique candidates
    unique_candidate_matches = list(set(possible_matches_index))
    possible_matches = source_gdf.iloc[unique_candidate_matches]

    # Conduct the actual intersect
    result = possible_matches.loc[possible_matches.intersects(intersecting_gdf.unary_union)]
    return result
########

In [14]:
def find_closest_stop(gdf_one_bui, gdf_stops, gdf_buff):
    uu_stops = gdf_stops.geometry.unary_union
    centr_bui = gdf_one_bui.geometry[0].centroid
    nrst_pt = nearest_points(centr_bui, uu_stops)[1]
    
    nearest_stop_id = list(gdf_stops[gdf_stops.geometry == nrst_pt]['stop_id'])[0]
    nearest_stop_geo = list(gdf_stops[gdf_stops.geometry == nrst_pt]['geometry'])[0]
    if gdf_buff.geometry[0].intersects(nearest_stop_geo):
        return [nearest_stop_id, nearest_stop_geo]
    else:
        return []

In [15]:
def get_closest_node(gdf_obj, linkid, graph_in_buff):
    line_geo = list(graph_in_buff[graph_in_buff.link_id == linkid]['geometry'])[0]
    start = Point(line_geo.coords[0])
    end = Point(line_geo.coords[-1])
    obj_centr = gdf_obj['geometry'][0]#.centroid

    if start.distance(obj_centr) < end.distance(obj_centr):
        return start
    else:
        return end
# 

In [16]:
def get_shrt_pth(gdf_one_bui, gdf_buff, gdf_stops, graph_ped):

    try:
        nearest_stop = find_closest_stop(gdf_one_bui, gdf_stops, gdf_buff)
    except:
        nearest_stop = []
    #
    if nearest_stop != []:
        gdf_near_stop = gpd.GeoDataFrame(data=[nearest_stop], columns=['stop_id', 'geometry'])
        gdf_near_stop.crs=graph_ped.crs
        graph_in_buff = intersect_using_spatial_index(graph_ped, gdf_buff[['geometry']])
        centr_bui = gdf_one_bui.copy()
        centr_bui.geometry = centr_bui.geometry.centroid

        text_trap = io.StringIO()
        with redirect_stdout(text_trap):
            closest_link_bui = mm.get_network_id(gdf_one_bui, graph_in_buff, 'link_id')[0]
            closest_link_stop = mm.get_network_id(gdf_near_stop, graph_in_buff, 'link_id')[0]

        closest_node_bui = get_closest_node(gdf_one_bui, closest_link_bui, graph_in_buff)
        closest_node_stop = get_closest_node(gdf_near_stop, closest_link_stop, graph_in_buff)

        one_G = mm.gdf_to_nx(graph_in_buff)
        res_lst = nx.shortest_path(one_G, closest_node_bui.coords[0], closest_node_stop.coords[0])

        if len(res_lst) == 1:
            line_path = LineString([closest_node_bui, closest_node_stop])
            path_lngth = 0
        elif len(res_lst) == 0:
            line_path = LineString([closest_node_bui, closest_node_stop])
            path_lngth = -1
        else:
            line_path = LineString(res_lst)
            path_lngth = 1
    # 
        return [nearest_stop[0], path_lngth, res_lst, line_path]
    else:
        return []

In [17]:
def generate_graph(edges, fields):
    G = nx.Graph()

    key = 0
    for index, row in edges.iterrows():
        first = row.from_node
        last = row.to_node

        data = [row[f] for f in fields]
        attributes = dict(zip(fields, data))
        G.add_edge(first, last, key=key, **attributes)
        key += 1
    # 
    return G

In [18]:
def get_lst_nodes(egg):
    lst_nodes_end=[]
    lst_nodes = list(egg.nodes())
    i=0
    for i in (range(len(lst_nodes))):
        one_node = lst_nodes[i]
        lst_neigh = list(nx.all_neighbors(egg, one_node))
        if len(lst_neigh) < 2:
            lst_nodes_end.append(one_node)
    #
    return lst_nodes_end

In [42]:
def get_strt_end_paths(egg, closest_node, lst_nodes_end):
    
    shortest_paths = list(nx.all_pairs_dijkstra_path(egg))
    
    start_ind=0
    i=0
    for i in range(len(shortest_paths)):
        if shortest_paths[i][0] == closest_node:
            start_ind = i
            break
    # 
    lst_paths_to_ends = []
    i=0
    for nd in shortest_paths[start_ind][1]:
        if nd in lst_nodes_end:
            lst_paths_to_ends.append(shortest_paths[start_ind][1][nd])
    # 
    try:
        lst_paths_to_ends.remove([closest_node])
    except:
        pass
    return lst_paths_to_ends

In [39]:
def find_closest_node(one_stop_gdf, nodes_in_buff):

    stop_pt = one_stop_gdf.geometry[0]
    np_nib = nodes_in_buff.to_numpy()
    ind_ndid = list(nodes_in_buff.columns).index('nodeID')
    ind_geo = list(nodes_in_buff.columns).index('geometry')

    closest_node = np_nib[0, ind_ndid]
    dstnc = 100000000
    i=0
    for i in range(len(np_nib)):
        node_pt = np_nib[i, ind_geo]
        if stop_pt.distance(node_pt) < dstnc:
            dstnc = stop_pt.distance(node_pt)
            closest_node = np_nib[i, ind_ndid]
    # 

    return closest_node

In [ ]:
def get_one_stop_sprut(one_stop_gdf, nodes, G):
    gdf_buff = one_stop_gdf.copy()
    gdf_buff.geometry = gdf_buff.geometry.to_crs('epsg:32640').buffer(600).to_crs('epsg:4326')
    nodes_in_buff = intersect_using_spatial_index(nodes, gdf_buff[['geometry']]).reset_index(drop=True)

    closest_node = find_closest_node(one_stop_gdf, nodes_in_buff)

    egg = nx.ego_graph(G, closest_node, radius=500, distance='length')
    edglst = nx.to_pandas_edgelist(egg)
    lst_nodes_end = get_lst_nodes(egg)
    lst_paths_to_ends = get_strt_end_paths(egg, closest_node, lst_nodes_end)
    return lst_paths_to_ends

In [20]:
graph_notdir = graph[graph.direction == 'direct'].reset_index(drop=True)
graph_notdir = graph_notdir[['link_id', 'from_node', 'to_node', 'geometry']]

In [21]:
graph_notdir_32640 = graph_notdir.copy()
graph_notdir_32640 = graph_notdir_32640.to_crs('epsg:32640')
graph_notdir_32640['length'] = graph_notdir_32640.geometry.length
fields = list(graph_notdir_32640.columns)

In [22]:
# https://github.com/martinfleis/momepy/blob/v0.3.0/momepy/utils.py#L117-L156
%time G = generate_graph(graph_notdir_32640, fields)

Wall time: 22.5 s


In [23]:
graph_notdir_32640.columns

Index(['link_id', 'from_node', 'to_node', 'geometry', 'length'], dtype='object')

In [43]:
i=0
for i in range(1):
# for i in range(len(gdf_stops)):
    one_stop_gdf = gdf_stops.iloc[[i]].reset_index(drop=True)
    %time lst_paths_to_ends = get_one_stop_sprut(one_stop_gdf, nodes, G)

Wall time: 714 ms


In [ ]:
# def get_last_link_list(lst_paths_to_ends, nodes, ):



In [44]:
lst_paths_to_ends

[[27397, 27394, 27395],
 [27397, 27415, 27658, 27657, 27656],
 [27397, 27415, 27658, 27657, 27732],
 [27397, 27394, 27416, 27400, 27399, 27402],
 [27397, 27415, 27658, 27495, 27496, 27554],
 [27397, 27415, 27658, 27495, 27754, 27824],
 [27397, 27415, 27658, 27495, 27754, 27753],
 [27397, 27415, 27658, 27522, 27712, 27737],
 [27397, 27386, 27385, 27413, 26931, 26930, 27038],
 [27397, 27415, 27658, 27522, 27258, 26951, 27458],
 [27397, 27415, 27658, 27522, 27712, 27817, 27864],
 [27397, 27415, 27658, 27522, 27258, 27036, 26603, 26846],
 [27397, 27415, 27658, 27522, 27258, 27036, 26953, 26925],
 [27397, 27386, 27385, 27413, 26931, 26930, 26929, 26898, 26899],
 [27397, 27415, 27658, 27522, 27258, 26951, 26952, 27232, 27233],
 [27397, 27386, 27385, 27413, 26931, 27396, 26908, 27546, 27542, 27543],
 [27397, 27386, 27385, 27413, 26931, 27396, 26908, 27347, 27204, 27161],
 [27397, 27386, 27385, 27413, 26931, 27396, 26908, 27347, 27204, 27346],
 [27397, 27386, 27385, 27413, 26931, 26930, 26929,

In [133]:
graph_notdir[((graph_notdir.from_node == 22976) & (graph_notdir.to_node == 23222))]

,link_id,from_node,to_node,geometry
30495,60991,22976,23222,"LINESTRING (50.19849 53.22491, 50.19836 53.225..."


In [ ]:
#  для обрезки:
# начало - всегда одно и то же
# найти концы - нет соседей?
# на концах найти следующее ребро, которого нет в эго
# построить кратчайший путь от начала до конца
# из последовательности сделать линию
# обрезать линию по длине